In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def date_range_dataframe(start_date,end_date,freq='1H', column_name='hours'):
    df = pd.DataFrame({column_name: pd.date_range(start_date, end_date, freq=freq, closed='left')})
    return df

In [3]:
def make_dummies(series):
    df = pd.get_dummies(series,drop_first=True)
    df.columns = list(map(lambda x: '{}_'.format(series.name) + str(x),list(df.columns)))
    return df

In [75]:
def year_sincos_array(n,start_date,end_date):
    start_date = pd.to_datetime(start_date)
    end_date   = pd.to_datetime(end_date)
    
    ## day distance
    first_date = pd.to_datetime('{year}-01-01'.format(year = start_date.year))
    day_of_the_year_start = (start_date - first_date).days
        
    ## year distance
    distance = (end_date - first_date).days
    
    ##
    start = (np.pi*2)*(day_of_the_year_start/365)
    end = np.pi*2*(distance/365)
    
    length = np.arange(start, end, (end-start)/n)
    df =  pd.concat([pd.Series(np.sin(length)),pd.Series(np.cos(length))],axis=1)
    df.columns = ['sin','cos']
    return df

In [165]:
def hour_sincos_array(hours_by_day, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date   = pd.to_datetime(end_date)
    ## ! year distance
    distance = (end_date - start_date).days

    start = 0
    end = np.pi*2*distance
    length = np.arange(start, end, np.pi*2/(hours_by_day))
    df =  pd.concat([pd.Series(np.sin(length)),pd.Series(np.cos(length))],axis=1)
    df.columns = ['sin','cos']
    return df

In [192]:
def hours_by_day(freq):
    freq = freq.upper()
    if freq == '1H':
        return 24
    if freq == '2H':
        return 12
    if freq == '3H':
        return 8
    if freq == '4H':
        return 6
    if freq == '6H':
        return 4
    if freq == '8H':
        return 3
    if freq == '12H':
        return 2

In [200]:
def time_series_dataframe(start_date,end_date,freq='1H',datetime_column_name='hours',
                          weekday_dummies=True,
                          month_dummies=True,
                          hour_dummies=True,
                          year_sin_cos=True,
                          hour_sin_cos=True):
    df = date_range_dataframe(start_date,end_date,freq=freq,column_name=datetime_column_name)
    shape = df.shape
    if weekday_dummies:
        df = pd.concat([df,make_dummies(df.iloc[:,0].dt.weekday)],axis=1)
    if month_dummies:
        df = pd.concat([df,make_dummies(df.iloc[:,0].dt.month)],axis=1)
    if hour_dummies:
        df = pd.concat([df,make_dummies(df.iloc[:,0].dt.hour)],axis=1)
    if year_sin_cos:
        df = pd.concat([df,year_sincos_array(shape[0],start_date,end_date)],axis=1)
    if hour_sin_cos:
        df = pd.concat([df,hour_sincos_array(hours_by_day(freq),start_date,pd.to_datetime(end_date))],axis=1)        
    return df

In [201]:
x = pd.to_datetime('2016-03-01')

In [202]:
time_series_dataframe('2016-03-01','2018-01-01','1H')

,hours,hours_1,hours_2,hours_3,hours_4,hours_5,hours_6,hours_2,hours_3,hours_4,...,hours_18,hours_19,hours_20,hours_21,hours_22,hours_23,sin,cos,sin,cos
0,2016-03-01 00:00:00,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.858764,0.512371,0.000000,1.000000
1,2016-03-01 01:00:00,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.859131,0.511755,0.258819,0.965926
2,2016-03-01 02:00:00,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.859498,0.511139,0.500000,0.866025
3,2016-03-01 03:00:00,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.859864,0.510522,0.707107,0.707107
4,2016-03-01 04:00:00,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.860230,0.509905,0.866025,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16099,2017-12-31 19:00:00,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0.013627,0.999907,-0.965926,0.258819
16100,2017-12-31 20:00:00,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0.014345,0.999897,-0.866025,0.500000
16101,2017-12-31 21:00:00,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0.015062,0.999887,-0.707107,0.707107
16102,2017-12-31 22:00:00,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0.015779,0.999876,-0.500000,0.866025
